In [539]:
import numpy as np
import itertools
import random

In [540]:
class LinearCode:  # класс линейных кодов

  def __init__(self, matrix):
    self.matrix = matrix
    self.lead = []
    self.H = []

  def ref(self,matrix):
    notZero = []
    for j in range(matrix.shape[1]):
      if 1 in matrix[:, j]:
        notZero = np.where(matrix[:, j] == 1)
        matrix[[0,notZero[0][0]]] = matrix[[notZero[0][0], 0]]
        for k in range(1, notZero[0].size):
          matrix[notZero[0][k], :] += matrix[0, :]
          matrix[notZero[0][k], :] = matrix[notZero[0][k], :] % 2
        break
    if matrix.shape[0] == 1:
      return
    self.ref(matrix[1:, :])
    return

  def rref(self):
    self.ref(self.matrix)
    j_lead = 0
    for i in range(1, self.matrix.shape[0]):
      notZero = np.where(self.matrix[i, :] == 1)
      if notZero[0].size ==0 :
        return self.matrix
      j_lead = notZero[0][0]
      for k in range(0,i):
        if self.matrix[k,j_lead]:
          self.matrix[k,:] += self.matrix[i,:]
          self.matrix[k,:] = self.matrix[k,:]%2
    return self.matrix

  def delete_null_rows(self):  # удаляем нулевые строки
    for i in range(0,self.matrix.shape[0]):
      if 1 not in self.matrix[i,:]:
        self.matrix = np.delete(self.matrix,i,axis=0)
    return self.matrix

  def count_k_n(self):  # задаем  n - число столбцов, k - число строк
    return(self.matrix.shape[1],self.matrix.shape[0])

  def search_lead(self):  # фиксируем ведубщие столбцы
    lead = []
    for i in range(0, self.matrix.shape[0]):
      notZero = np.where(self.matrix[i, :] == 1)
      if notZero[0].size ==0 :
        self.lead = lead
        return lead
      else:
        lead.append(notZero[0][0])
    self.lead = lead
    return lead

  def delete_lead_colums(self):  # удаляем ведющие столбцы матрицы
    lead_colums = self.lead
    for i in range(0,len(lead_colums)):
      self.matrix = np.delete(self.matrix,lead_colums[i]-i,axis = 1)
    return self.matrix


  def checking_matrix(self):  # создаем проверочную матрицу H
    check_matrix = []
    temp = np.eye(self.matrix.shape[1],dtype = int)
    k = 0
    m = 0
    for i in range(0, self.matrix.shape[0] + self.matrix.shape[1]):
      if i in self.lead:
        check_matrix.append(self.matrix[k])
        k += 1
      else:
        check_matrix.append(temp[m])
        m +=1
    check_matrix = np.array(check_matrix)
    self.H = check_matrix
    return check_matrix


  def generate_code_words_1(self):  # формируем кодовые слова, складывая все слова из порождающего множества
    zer = np.zeros(self.matrix.shape[1], dtype=int)
    final_word = set()
    final_word.add(tuple(zer.tolist()))
    for i in range(1, self.matrix.shape[0] + 1):
      comd_word = list(itertools.combinations(range(self.matrix.shape[0]), i))
      for comb in comd_word:
        word = np.zeros(self.matrix.shape[1], dtype=int)
        for j in comb:
          word += self.matrix[j, :]
        word %= 2
        final_word.add(tuple(word.tolist()))
    return np.array(list(final_word))

  def generate_code_words_2(self):  # формируем кодовые слова, беря все кодовые слова длины k и умножая на матрицу
    n,k = self.count_k_n()
    final_code = itertools.product(range(2), repeat=k)
    answer = []
    for code in (final_code):
      one_word = np.dot(code,self.matrix)%2
      answer.append(one_word)
    return answer





Функция для 2ЛР

In [541]:
def code_syndrom_error(u_f,n_f,k_f,H_f,G_f, count_error_f):

  #Сформировать кодовое слово длины n из слова длины k
  v_f = np.dot(u_f,G_f)%2
  print ("u: ", u_f)
  print("v: ", v_f)

  #Внести ошибку в сформированное слово
  i_f = np.random.choice(n_f, size=count_error_f, replace=False)
  #random.randint(0, n_f, count_error_f)
  print(i_f)
  for index in (i_f):
      v_f[index] = (v_f[index]+1)%2
  error_v_f = v_f
  print("Полученное кодовое слово с ошибкой: ", error_v_f)


  #Вычислить синдром
  syndrom_error_f = np.dot(error_v_f,H_f)%2
  print("Синдромы для этого кодового слова: ", syndrom_error_f)

  #Исправить ошибку с использованием таблицы синдромов

  error_index_f = np.where((H_f == syndrom_error_f).all(axis=1))[0]
  print("Индекс элемента, где была ошибка:",error_index_f)
  error_v_f[error_index_f] = (error_v_f[error_index_f]+1)%2
  true_v_f = error_v_f
  print ("Восстановленное правильное слово: ", true_v_f)

  #Убедиться в правильности полученного слова.
  final_u_f = true_v_f[:k_f]
  print("Исправленное декодированное сообщение: ", final_u_f)



# **Часть 1**
2.1. Сформировать порождающую матрицу линейного кода (7, 4, 3).

In [542]:
n = 7
k = 4
d = 3
I_k  = np.eye(k)

X = np.array([[1,1,1],
              [1,1,0],
              [1,0,1],
              [0,1,1]])

G = np.concatenate((I_k, X), axis=1)

print(G)

[[1. 0. 0. 0. 1. 1. 1.]
 [0. 1. 0. 0. 1. 1. 0.]
 [0. 0. 1. 0. 1. 0. 1.]
 [0. 0. 0. 1. 0. 1. 1.]]


2.2 Сформировать проверочную матрицу на основе порождающей.

In [543]:
I_n_k = np.eye(n-k)


H = np.concatenate((X, I_n_k), axis=0)

print(H)

[[1. 1. 1.]
 [1. 1. 0.]
 [1. 0. 1.]
 [0. 1. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


2.3 Сформировать таблицу синдромов для всех однократных
ошибок.

In [544]:
errors = np.eye(7)
syndroms = dict()
for i in range(6, -1, -1):
    syndroms[tuple(np.matmul(errors[i, :], H)%2)] = errors[i, :]
print(syndroms)

{(0.0, 0.0, 1.0): array([0., 0., 0., 0., 0., 0., 1.]), (0.0, 1.0, 0.0): array([0., 0., 0., 0., 0., 1., 0.]), (1.0, 0.0, 0.0): array([0., 0., 0., 0., 1., 0., 0.]), (0.0, 1.0, 1.0): array([0., 0., 0., 1., 0., 0., 0.]), (1.0, 0.0, 1.0): array([0., 0., 1., 0., 0., 0., 0.]), (1.0, 1.0, 0.0): array([0., 1., 0., 0., 0., 0., 0.]), (1.0, 1.0, 1.0): array([1., 0., 0., 0., 0., 0., 0.])}


2.4. Сформировать кодовое слово длины n из слова длины k. Внести
однократную ошибку в сформированное слово. Вычислить синдром,
исправить ошибку с использованием таблицы синдромов. Убедиться
в правильности полученного слова.


In [545]:
#Воспользуемся функцией
u_1 = np.array([1, 0, 0, 1])
code_syndrom_error(u_1,n,k,H,G,1)


u:  [1 0 0 1]
v:  [1. 0. 0. 1. 1. 0. 0.]
[2]
Полученное кодовое слово с ошибкой:  [1. 0. 1. 1. 1. 0. 0.]
Синдромы для этого кодового слова:  [1. 0. 1.]
Индекс элемента, где была ошибка: [2]
Восстановленное правильное слово:  [1. 0. 0. 1. 1. 0. 0.]
Исправленное декодированное сообщение:  [1. 0. 0. 1.]


2.5. Сформировать кодовое слово длины n из слова длины k. Внести
двукратную ошибку в сформированное слово. Вычислить синдром,
исправить ошибку с использованием таблицы синдромов.
Убедиться, что полученное слово отличается от отправленного

In [546]:
#Воспользуемся функцией
u_2 = np.array([1, 0, 0, 1])
code_syndrom_error(u_2,n,k,H,G,2)

u:  [1 0 0 1]
v:  [1. 0. 0. 1. 1. 0. 0.]
[6 1]
Полученное кодовое слово с ошибкой:  [1. 1. 0. 1. 1. 0. 1.]
Синдромы для этого кодового слова:  [1. 1. 1.]
Индекс элемента, где была ошибка: [0]
Восстановленное правильное слово:  [0. 1. 0. 1. 1. 0. 1.]
Исправленное декодированное сообщение:  [0. 1. 0. 1.]


# Часть 2

2.6. Сформировать порождающую матрицу линейного кода (n, k, 5).

In [547]:
n_2 = 12
k_2 = 4
d_2 = 5
I_k_2  = np.eye(k_2)

X_2 = np.array([[1, 1, 1, 1, 0, 0, 0, 0],
                [0, 0, 0, 0, 1, 1, 1, 1],
                [0, 0, 1, 1, 0, 0, 1, 1],
                [1, 0, 1, 0, 1, 0, 1, 0]])


G_2 = np.concatenate((I_k_2, X_2), axis=1)

print(G_2)

[[1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.]
 [0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1.]
 [0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0.]]


2.7 Сформировать проверочную матрицу на основе порождающей.

In [548]:
I_n_k_2 = np.eye(n_2-k_2)


H_2 = np.concatenate((X_2, I_n_k_2), axis=0)

print(H_2)

[[1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1.]
 [0. 0. 1. 1. 0. 0. 1. 1.]
 [1. 0. 1. 0. 1. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


2.8 Сформировать таблицу синдромов для всех однократных и
двукратных ошибок

In [549]:
errors_1 = np.eye(n_2)
syndroms_1 = dict()
for i in range(n_2-1, -1, -1):
    syndroms_1[tuple(np.matmul(errors_1[i, :], H_2)%2)] = errors_1[i, :]
print(syndroms_1)

error_2 = np.eye(n_2)
for i in range (n_2):
  num = np.random.randint(1, n_2, 1)
  if(num != i):
    error_2[i][num] = (error_2[i][num]+1)%2
  else:
    error_2[i][num-1] = (error_2[i][num-1]+1)%2


syndroms_2 = dict()
for i in range(n_2-1,-1,-1):
  syndroms_2[tuple(np.matmul(error_2[i, :], H_2)%2)] = error_2[i, :]
print(syndroms_2)

{(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0): array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]), (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0): array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]), (0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0): array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]), (0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0): array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]), (0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0): array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]), (0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0): array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]), (0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0): array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]), (1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0): array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]), (1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0): array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]), (0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0): array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), (0.0, 0.0

2.9. Сформировать кодовое слово длины n из слова длины k. Внести
однократную ошибку в сформированное слово. Вычислить синдром,
исправить ошибку с использованием таблицы синдромов. Убедиться
в правильности полученного слова.

In [550]:
#Воспользуемся функцией
u_3 = np.array([1, 0 , 0 , 1])
code_syndrom_error(u_3,n_2,k_2,H_2,G_2,1)

u:  [1 0 0 1]
v:  [1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0.]
[7]
Полученное кодовое слово с ошибкой:  [1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0.]
Синдромы для этого кодового слова:  [0. 0. 0. 1. 0. 0. 0. 0.]
Индекс элемента, где была ошибка: [7]
Восстановленное правильное слово:  [1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0.]
Исправленное декодированное сообщение:  [1. 0. 0. 1.]


2.10. Сформировать кодовое слово длины n из слова длины k. Внести
двукратную ошибку в сформированное слово. Вычислить синдром,
исправить ошибку с использованием таблицы синдромов. Убедиться
в правильности полученного слова.

In [551]:
#Воспользуемся функцией
u_4 = np.array([1, 0, 1, 0])
code_syndrom_error(u_4,n_2,k_2,H_2,G_2,2)

u:  [1 0 1 0]
v:  [1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1.]
[7 3]
Полученное кодовое слово с ошибкой:  [1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1.]
Синдромы для этого кодового слова:  [1. 0. 1. 1. 1. 0. 1. 0.]
Индекс элемента, где была ошибка: []
Восстановленное правильное слово:  [1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1.]
Исправленное декодированное сообщение:  [1. 0. 1. 1.]


2.1. Сформировать кодовое слово длины n из слова длины k. Внести
трёхкратную ошибку в сформированное слово. Вычислить синдром,
исправить ошибку с использованием таблицы синдромов.
Убедиться, что полученное слово отличается от отправленного

In [552]:
#Воспользуемся функцией
u_5 = np.array([1, 0, 1, 0])
code_syndrom_error(u_5,n_2,k_2,H_2,G_2,3)

u:  [1 0 1 0]
v:  [1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1.]
[11  7  4]
Полученное кодовое слово с ошибкой:  [1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0.]
Синдромы для этого кодового слова:  [1. 0. 0. 1. 0. 0. 0. 1.]
Индекс элемента, где была ошибка: []
Восстановленное правильное слово:  [1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0.]
Исправленное декодированное сообщение:  [1. 0. 1. 0.]
